In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
if not os.path.exists('./nfl_data/weekly/cbs_weekly'):
    os.mkdir('./nfl_data/weekly/cbs_weekly')

#### Scrapes CBS Weekly for all positions & all weeks
Saves files to `cbs_weekly` directory

In [3]:
positions = ['QB', 'WR', 'RB', 'TE', 'K']
weeks = [19, 20, 21, 23] # 22 was pro bowl

file_list = []

for position in positions:
    for week in weeks:
        url = f'https://www.cbssports.com/nfl/stats/leaders/live/{position}/{week}/'
        print(url)
        res = requests.get(url)
        print(res)
        output = open(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls', 'wb')
        output.write(res.content)
        file_list.append(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        output.close()
        data = pd.read_html(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        df = data[0]
        # df = df.droplevel(0, axis=1)
        print((position, week), df.shape)

https://www.cbssports.com/nfl/stats/leaders/live/QB/19/
<Response [200]>
('QB', 19) (13, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/20/
<Response [200]>
('QB', 20) (8, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/21/
<Response [200]>
('QB', 21) (3, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/23/
<Response [200]>
('QB', 23) (2, 11)
https://www.cbssports.com/nfl/stats/leaders/live/WR/19/
<Response [200]>
('WR', 19) (56, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/20/
<Response [200]>
('WR', 20) (39, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/21/
<Response [200]>
('WR', 21) (14, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/23/
<Response [200]>
('WR', 23) (10, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/19/
<Response [200]>
('RB', 19) (35, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/20/
<Response [200]>
('RB', 20) (19, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/21/
<Response [200]>


In [4]:
file_list

['./nfl_data/weekly/cbs_weekly/week-19_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-20_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-21_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-23_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-19_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-20_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-21_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-23_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-19_K.xls',
 './nfl_data/weekly/cbs_weekly/week-20_K.xls',
 './nfl_data/weekly/cbs_weekly/week-21_K.xls',
 './nfl_data/weekly/cbs_weekly/week-23_K.xls']

In [5]:
# get df started with the 1st file './nfl_data/week-19_QB.xls'.  This way we have something to merge to.

if 'QB' in file_list[0]:
    df_qb = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_QB.xls')
    df_qb = pd.DataFrame(df_qb[0])
    df_qb = df_qb.droplevel(0, axis=1)
    df_qb['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
    df_qb['Pos'] = [i.split()[-1] for i in df_qb['Player  Player on team']]
    df_qb.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
    df_qb['Player'] = df_qb['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
    df_qb

In [6]:
df_qb.shape

(13, 13)

In [7]:
for file in file_list[1:]:
    if 'QB' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_qb, df]
        df_qb = pd.concat([df_qb.squeeze() for df_qb in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_qb.shape}')

./nfl_data/weekly/cbs_weekly/week-20_QB.xls
./nfl_data/weekly/cbs_weekly/week-21_QB.xls
./nfl_data/weekly/cbs_weekly/week-23_QB.xls
The shape of the updated df is (26, 13)


In [8]:
df_qb.sort_values(by=['Player', 'Week'])

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
18,Aaron Rodgers,SF 13 - GB 10,9,20,29,225,0,0,—,—,—,20,QB
6,Ben Roethlisberger,KC 42 - PIT 21,20,29,44,215,2,0,2,-1,0,19,QB
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB
9,Derek Carr,CIN 26 - LV 19,16,29,54,310,1,1,1,20,0,19,QB
8,Jalen Hurts,TB 31 - PHI 15,17,23,43,258,1,2,8,39,0,19,QB
10,Jimmy Garoppolo,SF 23 - DAL 17,4,16,25,172,0,1,1,1,0,19,QB
20,Jimmy Garoppolo,SF 13 - GB 10,3,11,19,131,0,1,—,—,—,20,QB
23,Jimmy Garoppolo,LAR 20 - SF 17,19,16,30,232,2,1,1,4,0,21,QB
5,Joe Burrow,CIN 26 - LV 19,21,24,34,244,2,0,2,-2,0,19,QB
17,Joe Burrow,CIN 19 - TEN 16,11,28,37,348,0,1,2,5,0,20,QB


In [9]:
df_qb = df_qb.replace('—', 0) # get rid of the dashes, impute with 0
df_qb.head()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB
1,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB
2,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB
4,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB


In [10]:
# Attempt to convert all columns to int, those that fail are stored in list
numcols_to_change = df_qb.columns
numcols_to_change2 = []
for col in numcols_to_change:
    try:
        df_qb[col] = df_qb[col].astype(int)
        print('success!')
    except:
        numcols_to_change2.append(col)
        print(f'need to clean column: {col}')

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [11]:
max_week = df_qb['Week'].max()

df_qb.to_csv(f'./nfl_data/weekly/cbs_weekly/passing_through_week-{max_week}.csv', index=0)

In [12]:
df_qb.head()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB
1,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB
2,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB
3,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB
4,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB


In [13]:
# df_qb['Home_Team'] = df_qb['Game'].map(lambda score: score.split()[0])
# df_qb['Away_Team'] = df_qb['Game'].map(lambda score: score.split()[3])

In [14]:
# Team_Abbreviations_Dict = {
#     'Arizona Cardinals': 'ARI',
#     'Atlanta Falcons': 'ATL',
#     'Baltimore Ravens': 'BAL',
#     'Buffalo Bills' : 'BUF',
#     'Carolina Panthers': 'CAR',
#     'Chicago Bears': 'CHI',
#     'Cincinnati Bengals': 'CIN',
#     'Cleveland Browns': 'CLE',
#     'Dallas Cowboys': 'DAL',
#     'Denver Broncos': 'DEN',
#     'Detroit Lions': 'DET',
#     'Green Bay Packers': 'GB',
#     'Houston Texans': 'HOU',
#     'Indianapolis Colts': 'IND',
#     'Jacksonville Jaguars': 'JAX',
#     'Kansas City Chiefs': 'KC',
#     'Miami Dolphins': 'MIA',
#     'Minnesota Vikings': 'MIN',
#     'New England Patriots': 'NE',
#     'New Orleans Saints': 'NO',
#     'NY Giants': 'NYG',
#     'NY Jets': 'NYJ',
#     'Las Vegas Raiders': 'LV',
#     'Philadelphia Eagles': 'PHI',
#     'Pittsburgh Steelers': 'PIT',
#     'Los Angeles Chargers': 'LAC',
#     'San Francisco 49ers': 'SF',
#     'Seattle Seahawks': 'SEA',
#     'Los Angeles Rams': 'LAR',
#     'Tampa Bay Buccaneers': 'TB',
#     'Tennessee Titans': 'TEN',
#     'Washington Commanders': 'WAS'
# }

In [15]:
# def get_key(val):
#     for key, value in Team_Abbreviations_Dict.items():
#          if val == value:
#             return key
 
#     return "key doesn't exist"

In [16]:
# df_qb['Home_Team_Name_Full'] = df_qb['Home_Team'].map(get_key)
# df_qb['Away_Team_Name_Full'] = df_qb['Away_Team'].map(get_key)
# df_qb['Home_Team_Name_Mascot'] = df_qb['Home_Team_Name_Full'].map(lambda x: x.split()[-1])
# df_qb['Away_Team_Name_Mascot'] = df_qb['Away_Team_Name_Full'].map(lambda x: x.split()[-1])

In [17]:
# Read in weather df
weather_df = pd.read_csv('./nfl_data/weekly/weather/all_weeks_weather.csv')
weather_df.head()

,Away,Home,Forecast,Extended Forecast,Wind,Week,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,1,6,WSW,80,Humid and Partly Cloudy
1,Eagles,Falcons,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
2,Steelers,Bills,72f Mostly Cloudy,Mostly Cloudy. Rain overnight.,12m WSW,1,12,WSW,72,Mostly Cloudy
3,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,1,11,SW,82,Clear
4,49ers,Lions,DOME,Mostly Cloudy. Mostly cloudy throughout the day.,0,1,0,0,0,0


In [18]:
# Read in players
passing_df = pd.read_csv('./clean_data/passing.csv')
passing_df = passing_df[['Player', 'Tm_Abr', 'Team']]
passing_df.head()

,Player,Tm_Abr,Team
0,Tom Brady,TAM,Tampa Bay Buccaneers
1,Justin Herbert,LAC,Los Angeles Chargers
2,Matthew Stafford,LAR,Los Angeles Rams
3,Patrick Mahomes,KAN,Kansas City Chiefs
4,Derek Carr,LVR,Las Vegas Raiders


In [19]:
# Get Tm Abr and Team
df = pd.merge(df_qb, passing_df, on = 'Player', how = 'inner')
df.head()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,Tm_Abr,Team
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB,BUF,Buffalo Bills
1,Josh Allen,KC 42 - BUF 36,45,27,37,329,4,0,11,68,0,20,QB,BUF,Buffalo Bills
2,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB,KAN,Kansas City Chiefs
3,Patrick Mahomes,KC 42 - BUF 36,45,33,44,378,3,0,7,69,1,20,QB,KAN,Kansas City Chiefs
4,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB,LAR,Los Angeles Rams


In [20]:
# Create Mascot column
df['Mascot'] = df['Team'].map(lambda x: x.split()[-1])
df.sort_values(by='Week', ascending=True).head(10)

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,Tm_Abr,Team,Mascot
0,Josh Allen,BUF 47 - NE 17,48,21,25,308,5,0,6,66,0,19,QB,BUF,Buffalo Bills,Bills
23,Mitchell Trubisky,BUF 47 - NE 17,0,0,0,0,0,0,3,-2,0,19,QB,BUF,Buffalo Bills,Bills
2,Patrick Mahomes,KC 42 - PIT 21,46,30,39,404,5,1,3,29,0,19,QB,KAN,Kansas City Chiefs,Chiefs
22,Kyler Murray,LAR 34 - ARI 11,3,19,34,137,0,2,2,6,0,19,QB,ARI,Arizona Cardinals,Cardinals
4,Matthew Stafford,LAR 34 - ARI 11,28,13,17,202,2,0,6,22,1,19,QB,LAR,Los Angeles Rams,Rams
19,Jimmy Garoppolo,SF 23 - DAL 17,4,16,25,172,0,1,1,1,0,19,QB,SFO,San Francisco 49ers,49ers
18,Derek Carr,CIN 26 - LV 19,16,29,54,310,1,1,1,20,0,19,QB,LVR,Las Vegas Raiders,Raiders
8,Dak Prescott,SF 23 - DAL 17,22,23,43,254,1,1,4,27,1,19,QB,DAL,Dallas Cowboys,Cowboys
9,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,19,QB,TAM,Tampa Bay Buccaneers,Buccaneers
17,Jalen Hurts,TB 31 - PHI 15,17,23,43,258,1,2,8,39,0,19,QB,PHI,Philadelphia Eagles,Eagles


In [21]:
# Change TB12 to Week 1 for upcoming merge testing
df.at[9,'Week']=1 # Change a week to ensure the join is working properly

In [22]:
df_merged = pd.merge(df, weather_df, left_on = ['Mascot', 'Week'], right_on = ['Home', 'Week'], how = 'inner')
df_merged

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos,Tm_Abr,Team,Mascot,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Tom Brady,TB 31 - PHI 15,22,29,37,271,2,0,0,0,0,1,QB,TAM,Tampa Bay Buccaneers,Buccaneers,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,6,WSW,80,Humid and Partly Cloudy


In [23]:
# Reorganize columns
df_merged = df_merged[['Week', 'Player', 'Pos', 'Team', 'Tm_Abr', 'Mascot', 'Game', 'Away', 'Home', 'Forecast', 'Extended Forecast',
       'Wind', 'Wind_Speed_MPH', 'Wind_Direction', 'Temp', 'Weather_Desc', 'CMP  Pass Completions',
       'ATT  Pass Attempts', 'YDS  Passing Yards', 'TD  Touchdown Passes',
       'INT  Interceptions Thrown', 'ATT  Rushing Attempts',
       'YDS  Rushing Yards', 'TD  Rushing Touchdowns', 'FPTS  Fantasy Points',]]

In [24]:
# Create Away_Game boolean based on Away column matching Player Mascot column
df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Mascot'], 1, 0)
df_merged

,Week,Player,Pos,Team,Tm_Abr,Mascot,Game,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,FPTS Fantasy Points,Away_Game
0,1,Tom Brady,QB,Tampa Bay Buccaneers,TAM,Buccaneers,TB 31 - PHI 15,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,6,WSW,80,Humid and Partly Cloudy,29,37,271,2,0,0,0,0,22,0


In [25]:
# Test case - changes record to Away = Buccaneers, re-runs the boolean column calc. If you run this, run the next cell to return things to normal
df_merged.at[0,'Away']='Buccaneers'
df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Mascot'], 1, 0)
df_merged

,Week,Player,Pos,Team,Tm_Abr,Mascot,Game,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,FPTS Fantasy Points,Away_Game
0,1,Tom Brady,QB,Tampa Bay Buccaneers,TAM,Buccaneers,TB 31 - PHI 15,Buccaneers,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,6,WSW,80,Humid and Partly Cloudy,29,37,271,2,0,0,0,0,22,1


In [26]:
df_merged.at[0,'Away']='Cowboys'
df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Mascot'], 1, 0)

In [27]:
# I think this is our best version of the QB df. I didn't write it out because we only have one fake week where we'll actually have data until the season starts.
df_merged

,Week,Player,Pos,Team,Tm_Abr,Mascot,Game,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,FPTS Fantasy Points,Away_Game
0,1,Tom Brady,QB,Tampa Bay Buccaneers,TAM,Buccaneers,TB 31 - PHI 15,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,6,WSW,80,Humid and Partly Cloudy,29,37,271,2,0,0,0,0,22,0
